In [2]:
import numpy as np
import scipy as sp
import statsmodels.api as sm
from patsy import dmatrices
import pandas as pd
from pandas_datareader.data import DataReader
import plotly.express as px
import datetime as dt

In [10]:
#cpi = DataReader(["CPIAUCNS","CPILFESL"], "fred", start="1971-01", end="2018-12")
cpi = DataReader(["CPIAUCNS","CPILFESL"], "fred", start  = "1913-01")

In [11]:
cpi.head()

,CPIAUCNS,CPILFESL
DATE,,
1913-01-01,9.8,NaN
1913-02-01,9.8,NaN
1913-03-01,9.8,NaN
1913-04-01,9.8,NaN
1913-05-01,9.7,NaN


In [100]:
from fredapi import Fred
fred_api = 'd03c717886677acb9d63ff80b0ce10c0'
core_cpi_idx = 'CPILFESL'
head_cpi_idx = 'CPIAUCNS'
be_5 = 'T5YIE'
ry_5 = 'DFII5'
y_5 = 'DGS5'
fed_funds = 'FEDFUNDS'
fred = Fred(api_key=fred_api)
head_cpi = fred.get_series(head_cpi_idx)
core_cpi = fred.get_series(core_cpi_idx)
be5_data = fred.get_series(be_5)
ry5_data = fred.get_series(ry_5)
y5_data = fred.get_series(y_5)
ffr_data = pd.DataFrame({'ffr': fred.get_series(fed_funds)})

In [3]:
# Calculate 5-year moving average YoY core CPI
# Compare with 5-year break even
# Study the number of lags/leads between FFR and 5-year rates - do rates move before tightening?
df = pd.DataFrame([])
df['cpi'] = head_cpi
df['cpi_yoy'] = 100*df['cpi'].diff(12)/df['cpi']
df['cpi_lag_60'] = df['cpi'].shift(60)
df['cpi_5y'] = 100*((df['cpi']/df['cpi_lag_60'])**(1/5)-1)

In [4]:
be_df = pd.DataFrame([])
be_df['be_5'] = be5_data.dropna()
be_mon = be_df.dropna().resample("M").agg(lambda x: x[-1])
be5 = be_mon.set_index(be_mon.index.to_period('M').to_timestamp())

ry_df = pd.DataFrame([])
ry_df['ry_5'] = ry5_data.dropna()
ry_mon = ry_df.dropna().resample("M").agg(lambda x: x[-1])
ry5 = ry_mon.set_index(ry_mon.index.to_period('M').to_timestamp())

y_df = pd.DataFrame([])
y_df['y_5'] = y5_data.dropna()
y_mon = y_df.dropna().resample("M").agg(lambda x: x[-1])
y5 = y_mon.set_index(y_mon.index.to_period('M').to_timestamp())

yld_data = ry5.join(df, how = 'right')
yld_data = yld_data.join(y5, how = 'left')
yld_data['est_ry5'] = yld_data['y_5']-yld_data['cpi_5y']

In [148]:
g_dat = yld_data.loc['19610101':'20211130']
fig = px.line(g_dat, x = g_dat.index, y = ['ry_5','est_ry5'])
fig.show()

In [74]:
data0 = yld_data.loc['20100101':'20211231']
data1 = yld_data.loc['20030101':'20211231']
y, X = dmatrices('ry_5 ~ est_ry5', data = data0, return_type = 'dataframe')
reg = sm.OLS(y,X)
res = reg.fit()

In [71]:
x_predict = yld_data.loc['19620101':'20021231', 'est_ry5']

In [80]:
ry5_hat = pd.DataFrame({'ry_5': res.params[0] + res.params[1]*x_predict})

In [102]:
ry_long = pd.DataFrame({'ry_5': ry5_hat.append(data1)['ry_5']})
ry_long = ry_long.join(ffr_data, how = 'left')

In [103]:
fig1 = px.line(ry_long)
fig1.show()

,ry_5,ffr
1962-01-01,1.725280,2.15
1962-02-01,1.561789,2.37
1962-03-01,1.495556,2.85
1962-04-01,1.522712,2.78
1962-05-01,1.593470,2.36
...,...,...
2021-06-01,-1.600000,0.08
2021-07-01,-1.870000,0.10
2021-08-01,-1.710000,0.09
2021-09-01,-1.530000,0.08


In [96]:
ry_long.head()

1962-01-01    1.725280
1962-02-01    1.561789
1962-03-01    1.495556
1962-04-01    1.522712
1962-05-01    1.593470
Name: ry_5, dtype: float64